### Try to use the package scikit learn on chr22 with real phenotype
Test whether the result of using chr22 snps to do predict how is the correlaton. 

In [1]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import numpy as np
import pandas as pd
np.random.seed(1234)

In [12]:
chr_22 = pd.read_csv("/home/mcb/users/yzhu439/RA_Project/data/DREAM_RA_Responders_DosageData/Training_chr22.dos", sep=" ", header=None)
# transform the dataframe to array of shape patient by SNPs
chr22_array = chr_22.drop([0,2,3,4,5], axis=1).T

In [13]:
# convert extracted columns of dataframe to numpy array -> X data
chr22_array.columns = chr22_array.iloc[0]
dos_chr22 = chr22_array[1:]
dos_chr22_array = dos_chr22.values

In [14]:
# extract Response.deltaDAS from clinical text file as our Y data
pheno_df = pd.read_csv("/home/mcb/users/yzhu439/RA_Project/data/DREAM_RA_Responders_PhenoCov_Full.txt", sep=" ")
pheno = pheno_df['Response.deltaDAS'].values

###  first try fit lasso without adding covariates as additional features


In [15]:
# first split the data as 80% training and 20% testing
X_train, X_test, y_train, y_test = train_test_split(dos_chr22_array, pheno, test_size=0.2, random_state=10)

# initialize lasso model and start training
lasso_model = Lasso(alpha=0.1)
lasso_model.fit(X_train, y_train)

# evaluation by MSE
y_pred = lasso_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")


r_squared = r2_score(y_test, y_pred)
print(f"R-squared: {r_squared}")

Mean Squared Error: 2.4119915003760917
R-squared: -0.005537425180190292


###  first try fit lasso with covariates added as additional features


In [16]:
# first read in the covariates file
cov_df = pd.read_csv("/home/mcb/users/yzhu439/RA_Project/data/Dosage_for_PCA/chr22_covariates.csv", sep="\t")
cov_df_filter = cov_df[['baselineDAS', 'Age', 'Gender']]
cov = cov_df_filter.values


In [17]:
# some patients do not have age or sex information
# Find the row indices where NaN values are present
nan_row_indices = np.any(np.isnan(cov), axis=1)
ambiguous_pt = np.where(nan_row_indices)[0]


# remove these ambiguous patients from cov and the patient by SNPs array
cov_filtered = np.delete(cov, ambiguous_pt, axis=0)
chr22_array_filtered = np.delete(dos_chr22_array, ambiguous_pt, axis=0)

# now combine the two, treating covariates as additional features
chr22_withCOV = np.hstack((cov_filtered, chr22_array_filtered))

# delete these ambiguous patients from pheno array
pheno_filtered = np.delete(pheno, ambiguous_pt, axis=0)

In [18]:
print(chr22_withCOV.shape)
print(pheno_filtered.shape)

(2417, 33748)
(2417,)


In [19]:
# first split the data as 80% training and 20% testing
X_train, X_test, y_train, y_test = train_test_split(chr22_withCOV, pheno_filtered, test_size=0.2, random_state=10)

# initialize lasso model and start training
lasso_model = Lasso(alpha=0.1)
lasso_model.fit(X_train, y_train)

# evaluation by MSE
y_pred = lasso_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

r_squared = r2_score(y_test, y_pred)

print(f"R-squared: {r_squared}")

Mean Squared Error: 1.9234603878062995
R-squared: 0.19066598405817292


### recreate covariates file without header

In [3]:
# read out the old covariates file
cov_df = pd.read_csv("/home/mcb/users/yzhu439/RA_Project/data/Dosage_for_PCA/chr22_covariates.csv", sep="\t")
cov_df

# save it setting header=None


,FID,IID,baselineDAS,Age,Gender,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10
0,0,1877000,3.18,54.02000,0.0,-9.776815,-4.369307,6.492682,1.925455,4.963029,9.583803,8.655424,-2.315439,3.745552,-5.999781
1,0,612000,4.65,61.08333,1.0,5.016928,-5.070574,1.122143,-3.886903,0.968361,-12.830465,-10.162821,2.112627,-9.952401,-3.008695
2,0,111000,4.38,58.00000,1.0,-8.579379,-1.904669,-10.032267,-2.863686,0.032414,-7.396252,-1.416718,-4.039121,3.328055,4.399511
3,0,1780000,2.45,51.07000,1.0,0.532981,2.563628,-12.603776,6.183865,-4.458032,-5.209152,11.484811,6.186862,1.504132,-3.730175
4,0,825000,3.87,73.00000,1.0,-0.954074,-6.798254,1.561729,-3.836749,21.355867,2.199160,-12.792151,-3.305315,-4.329553,2.847404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2701,0,2732000,5.19,NaN,1.0,-4.952797,2.184384,-12.894696,-2.084022,-4.427039,12.657665,-5.903688,-0.860558,3.623311,2.453478
2702,0,2736000,6.59,NaN,1.0,19.286150,6.393782,0.355857,10.367995,1.848236,4.195206,-1.758485,-1.104616,-4.396043,-0.997822
2703,0,2744000,6.74,NaN,1.0,-6.515565,19.041983,1.455983,3.081210,-4.516184,-9.596668,7.567928,-14.530116,11.167546,-1.686096
2704,0,2748000,5.01,NaN,1.0,7.794618,-4.880688,-1.342529,-0.745498,7.093612,11.159411,8.589460,-0.719585,-1.534191,2.341495
